In [122]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models

## ResNet50 From Scratch

In [123]:
class Bottleneck(nn.Module):
    def __init__(self, in_channels, intermediate_channels, stride=1, downsample = None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, intermediate_channels, kernel_size = 1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        
        self.conv2 = nn.Conv2d(intermediate_channels, intermediate_channels, kernel_size = 3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        
        self.conv3 = nn.Conv2d(intermediate_channels, intermediate_channels*4, kernel_size = 1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(intermediate_channels*4)

        self.downsample = downsample
        self.relu = nn.ReLU()
    
    def forward(self, x):
        clone = x.clone()

        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))

        if self.downsample is not None:
            clone = self.downsample(clone)
        
        x = x + clone
        x = self.relu(x)
        return x

class ResNet50(nn.Module):
    def __init__(self, num_classes, num_channels=3):
        super().__init__()
        self.conv1 = nn.Conv2d(num_channels, 64, 7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        layer_channels = [64, 128, 256, 512]
        layer_repeats = [3, 4, 6, 3]
        self.layer1 = self.layer(64, layer_channels[0], layer_repeats[0], stride = 1)
        self.layer2 = self.layer(256, layer_channels[1], layer_repeats[1], stride = 2)
        self.layer3 = self.layer(512, layer_channels[2], layer_repeats[2], stride = 2)
        self.layer4 = self.layer(1024, layer_channels[3], layer_repeats[3], stride = 2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(2048, num_classes)
    
    def layer(self, in_channels, intermediate_channels, layer_repeats, stride):
        layers = []
        downsample = None
        if stride != 1 or in_channels != intermediate_channels*4:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, intermediate_channels*4, kernel_size = 1, stride = stride),
                nn.BatchNorm2d(intermediate_channels*4)
            )

        layers.append(
                Bottleneck(in_channels, intermediate_channels, stride, downsample)
            )
        for _ in range(layer_repeats-1):
            layers.append(
                Bottleneck(intermediate_channels*4, intermediate_channels)
            )
        
        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.shape[0],x.shape[1])
        x = self.fc(x)

        return x

In [124]:
random_tensor = torch.rand(3, 3, 224, 224)
model = ResNet50(10)
test = model(random_tensor)

In [125]:
test

tensor([[ 1.3789e+00,  6.4588e-01, -2.4217e-01,  8.8851e-01, -5.7838e-01,
         -1.2431e+00, -2.2273e+00,  1.2140e+00,  2.0840e-03,  6.2252e-01],
        [ 1.2648e+00,  6.0156e-01, -3.3839e-02,  7.6550e-01, -6.9319e-01,
         -1.1931e+00, -2.1116e+00,  1.2221e+00, -2.4767e-03,  6.5630e-01],
        [ 1.2938e+00,  6.1952e-01, -1.3998e-01,  7.2315e-01, -5.9283e-01,
         -1.2762e+00, -2.1429e+00,  1.2448e+00,  1.0386e-02,  6.4930e-01]],
       grad_fn=<AddmmBackward0>)